# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
pd_readcsv_df = pd.read_csv("../WeatherPy/cities.csv")
pd_readcsv_df.head()


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)


In [ ]:
locations = pd_readcsv_df[['Latitude', 'Longitude']]
humidity = pd_readcsv_df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 200, point_radius = 3, dissipating = False)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.


  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [ ]:
weather_crit_df = pd_readcsv_df.loc[(pd_readcsv_df['Max Temp'] < 80) & (pd_readcsv_df['Max Temp'] > 75) &
                                    (pd_readcsv_df['Wind Speed'] < 10) &
                                    (pd_readcsv_df['Cloudiness'] == 0)]
weather_crit_df.dropna()
weather_crit_df


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = weather_crit_df

In [ ]:
hotel_df["Hotel Name"] = ""
hotel_df_clean = hotel_df[["City", "Latitude", "Longitude", "Country", "Hotel Name"]]
hotel_df_clean.head()




In [ ]:
params = {'radius' : 5000,
          'types' : "hotel",
          'key' : g_key}

for index, row in hotel_df_clean.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    params["location"] = f"{latitude},{longitude}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    address = requests.get(base_url, params = params).json()
    try:
        hotel_df_clean.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except: 
        print("Hotel not found")
                   

                   
    

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
fig2 = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig2.add_layer(marker_layer)


# Display figure
fig2